## 1. 모델 불러오기

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# DeepSeek Math 모델 및 토크나이저 로드
model_name = "deepseek-ai/deepseek-math-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.6k [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

Logits shape: torch.Size([1, 9, 102400])
Predicted next token ID: 185
Predicted next token: 



## 2. 토크나이저로 token id, attention mask 로 변환하기

In [ ]:
# 예제 입력 문장
text = "What is the integral of x squared?"

# 1. 텍스트를 토큰으로 변환 (tokenize)
tokens = tokenizer.tokenize(text)
print("Tokenized output:", tokens)

# 2. 토큰을 토큰 ID로 변환 (encode)
token_ids = tokenizer.encode(text, add_special_tokens=True)
print("Token IDs:", token_ids)

# 3. 토큰 ID와 attention mask 생성 (return_tensors="pt"로 PyTorch 텐서 변환)
inputs = tokenizer(text, return_tensors="pt")
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# 4. 변환된 데이터 출력
print("Input IDs (Tensor):", input_ids)
print("Attention Mask (Tensor):", attention_mask)

# 5. 디코딩 (토큰 ID를 다시 원래 텍스트로 변환)
decoded_text = tokenizer.decode(input_ids[0])
print("Decoded Text:", decoded_text)

Tokenized output: ['What', 'Ġis', 'Ġthe', 'Ġintegral', 'Ġof', 'Ġx', 'Ġsquared', '?']
Token IDs: [100000, 2640, 317, 254, 12667, 280, 1376, 37160, 30]
Input IDs (Tensor): tensor([[100000,   2640,    317,    254,  12667,    280,   1376,  37160,     30]])
Attention Mask (Tensor): tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])
Decoded Text: <｜begin▁of▁sentence｜>What is the integral of x squared?


## 3. 모델의 첫번째 가중치에 곱해서 token id를 attention mask로 바꾸기


In [ ]:
# 입력 문장
text = "What is the integral of x squared?"

# 토큰화 및 텐서 변환
inputs = tokenizer(text, return_tensors="pt")
input_ids = inputs["input_ids"].to(model.device)

# 모델의 임베딩 레이어 추출
embedding_layer = model.get_input_embeddings()  # 모델의 input embedding 층

# 토큰 ID를 워드 임베딩 벡터로 변환
word_embeddings = embedding_layer(input_ids)  # (batch_size, sequence_length, embedding_dim)

# 출력 크기 확인
print("Word Embeddings Shape:", word_embeddings.shape)

# 첫 번째 토큰의 임베딩 벡터 출력
print("First Token Embedding (begin_of_sentence):")
print(word_embeddings[0, 0, :].detach().cpu().numpy())

# 두 번째 토큰 (What)의 임베딩 벡터 출력
print("Second Token Embedding (What):")
print(word_embeddings[0, 1, :].detach().cpu().numpy())

Word Embeddings Shape: torch.Size([1, 9, 4096])
First Token Embedding (begin_of_sentence):
[-0.0282    0.02307   0.4941   ...  0.01843  -0.007385  0.01306 ]
Second Token Embedding (What):
[ 0.0376 -0.0464  0.4395 ... -0.1216 -0.1279 -0.1748]


`Word Embeddings Shape: torch.Size([1, 9, 4096])`  (배치 크기=1, 시퀀스 길이=9, 임베딩 차원=4096)

## 주요 LLM의 임베딩 차원 비교

### 주요 언어 모델의 임베딩 차원 비교

| 모델 | 파라미터 수 | 임베딩 차원 |
|------|------------|------------|
| **DeepSeek-R1** | 70B | 4096 |
| **DeepSeek-R1 Distill Qwen 14B** | 14B | 4096 |
| **Qwen 2.5 Coder 32B Instruct** | 32B | 8192 |
| **Gemma-2 Instruct** | 27B | 8192 |
| **Mistral Small 3 Instruct** | 24B | 8192 |
| **Mixtral-8x7B-Instruct-v01** | 56B (8x7B) | 8192 |
| **GPT-3** | 175B | 12,288 |
| **GPT-4** | ? | ? (추정 12,288 이상) |


* 같은 구조의 모델이라도 학습 데이터, 초기화 방식, 최적화 과정이 다르면 **임베딩 행렬도 다르게 학습**됩니다.
* Distilled 모델, Mixture of Experts 모델, Efficient Transformer 모델에서는 **임베딩 공유**